# Steady state solutions

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: GlcConst, G3P, Pyr, NADH_c, NADH_m, ATP_c, ADP_c, AMP_c, Ca_m, Ca_c, x, ΔΨm, degavg
import MitochondrialDynamics: second, μM, mV, mM, Hz
import PyPlot as plt
rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["font.size"] = 14
# rcParams["font.sans-serif"] = "Arial"
# rcParams["font.family"] = "sans-serif"

## Figure 2

Steady-state solutions across a range of glucose levels.

In [ ]:
@named sys = make_model()

In [ ]:
pidx = Dict(k => i for (i, k) in enumerate(parameters(sys)))
idxGlc = pidx[GlcConst]

function remake_glc(prob, g)
    p = copy(prob.p)
    p[idxGlc] = g
    remake(prob; p=p)
end

In [ ]:
prob = SteadyStateProblem(sys, [])
# Empty array = using default u0

glc = range(3.0mM, 30.0mM, length=101)  # Range of glucose

sols = map(glc) do g
    solve(remake_glc(prob, g), alg=DynamicSS(Rodas5()))
end;

In [ ]:
extract(sols, k) = map(s->s[k], sols)

function plot_fig2(glc, sols; figsize=(12, 12))
    glc5 = glc ./ 5
    g3p = extract(sols, G3P) .* 1000
    pyr = extract(sols, Pyr) .* 1000
    ca_c = extract(sols, Ca_c) .* 1000
    ca_m = extract(sols, Ca_m) .* 1000
    nadh_c = extract(sols, NADH_c) .* 1000
    nadh_m = extract(sols, NADH_m) .* 1000
    atp_c = extract(sols, ATP_c) .* 1000
    adp_c = extract(sols, ADP_c) .* 1000
    amp_c = extract(sols, AMP_c) .* 1000
    dpsi = extract(sols, ΔΨm) .* 1000
    x1 = extract(sols, x[1])
    x2 = extract(sols, x[2])
    x3 = extract(sols, x[3])
    deg = extract(sols, degavg)

    fig, ax = plt.subplots(3, 3; figsize)

    ax[1, 1].plot(glc5, g3p)
    ax[1, 1].set(title="(A) G3P (μM)", ylim=(0.0, 10.0))
    ax[1, 2].plot(glc5, pyr)
    ax[1, 2].set(title="(B) Pyruvate (μM)", ylim=(0.0, 120.0))
    ax[1, 3].plot(glc5, ca_c, label="cyto")
    ax[1, 3].plot(glc5, ca_m, label="mito")
    ax[1, 3].legend()
    ax[1, 3].set(title="(C) Calcium (μM)", ylim=(0.0, 1.5))
    ax[2, 1].plot(glc5, nadh_c, label="cyto")
    ax[2, 1].plot(glc5, nadh_m, label="mito")
    ax[2, 1].legend()
    ax[2, 1].set(title="(D) NADH (μM)")
    ax[2, 2].plot(glc5, atp_c, label="ATP")
    ax[2, 2].plot(glc5, adp_c, label="ADP")
    ax[2, 2].plot(glc5, amp_c, label="AMP")
    ax[2, 2].legend()
    ax[2, 2].set(title="(E) Adenylates (μM)")
    ax[2, 3].plot(glc5, atp_c ./ adp_c)
    ax[2, 3].set(title="(F) ATP/ADP ratio")
    ax[3, 1].plot(glc5, dpsi, label="cyto")
    ax[3, 1].set(title="(G) ΔΨ (mV)", ylim=(80, 160), xlabel="Glucose (X)")
    ax[3, 2].plot(glc5, x1, label="X1")
    ax[3, 2].plot(glc5, x2, label="X2")
    ax[3, 2].plot(glc5, x3, label="X3")
    ax[3, 2].set(title="(H) Mitochondrial nodes", xlabel="Glucose (X)")
    ax[3, 3].plot(glc5, deg)
    ax[3, 3].set(title="(I) Average Node Degree", xlabel="Glucose (X)")

    for a in ax
        a.set_xticks(1:6)
        a.grid()
    end

    plt.tight_layout()
    return fig
end

In [ ]:
fig2 = plot_fig2(glc, sols)

In [ ]:
# Uncomment if pdf file is required
fig2.savefig("Fig2.tif", dpi=300, format="tiff", pil_kwargs=Dict("compression" => "tiff_lzw"))

## Figure 3

Steady-state solutions accross a range and glucose concentrations and enzymic levels.

In [ ]:
using MitochondrialDynamics: GlcConst, G3P, Pyr, NADH_c, NADH_m, ATP_c, ADP_c, AMP_c, Ca_m, Ca_c, x, ΔΨm, degavg, VmaxF1, VmaxETC, pHleak

In [ ]:
@named sys = make_model()
prob = SteadyStateProblem(sys, [])

pidx = Dict(k => i for (i, k) in enumerate(parameters(sys)))
idxGlc = pidx[GlcConst]

function solve_fig3(glc, r, protein, prob, alg=DynamicSS(Rodas5()))
    idx = pidx[protein]
    p = copy(prob.p)
    p[idxGlc] = glc
    p[idx] = prob.p[idx] * r
    return solve(remake(prob, p=p), alg)
end

In [ ]:
rGlc1 = LinRange(3.0, 30.0, 50)
rGlc2 = LinRange(4.0, 30.0, 50)
rF1 = LinRange(0.1, 2.0, 50)
rETC = LinRange(0.1, 2.0, 50)
rHL = LinRange(0.1, 5.0, 50)

uInf_f1 = [solve_fig3(glc, r, VmaxF1, prob) for r in rF1, glc in rGlc1]
uInf_etc = [solve_fig3(glc, r, VmaxETC, prob) for r in rETC, glc in rGlc1]
uInf_hl = [solve_fig3(glc, r, pHleak, prob) for r in rHL, glc in rGlc2];

In [ ]:
function plot_fig3(;
    figsize=(13, 10),
    levels=40,
    cmaps=["bwr", "magma", "viridis"],
    ylabels=[
        "ATP synthase capacity (X)",
        "ETC capacity (X)",
        "Proton leak rate (X)"
    ],
    cbarlabels=["<k>", "ΔΨ", "ATP/ADP"],
    xxs=(rGlc1, rGlc1, rGlc2),
    xscale=5.0,
    yys=(rF1, rETC, rHL),
    zs=(uInf_f1, uInf_etc, uInf_hl),
    extremes=((1.0, 2.0), (80.0, 180.0), (0.0, 60.0))
)
    # mapping functions
    fs = (s -> s[degavg], s -> s[ΔΨm] * 1000, s -> s[ATP_c] / s[ADP_c])

    fig, axes = plt.subplots(3, 3; figsize)

    for col in 1:3
        f = fs[col]
        cm = cmaps[col]
        cbl = cbarlabels[col]
        vmin, vmax = extremes[col]

        # lvls = LinRange(vmin, vmax, levels)
        for row in 1:3
            xx = xxs[row] ./ xscale
            yy = yys[row]
            z = zs[row]
            ax = axes[row, col]

            ylabel = ylabels[row]

            mesh = ax.pcolormesh(
                xx, yy, map(f, z);
                shading="gouraud",
                rasterized=true,
                # levels=levels,
                cmap=cm, 
                vmin=vmin, 
                vmax=vmax
            )

            ax.set(ylabel=ylabel, xlabel="Glucose (X)")

            # Arrow annotation
            # https://matplotlib.org/stable/tutorials/text/annotations.html#plotting-guide-annotation
            if row == 1
                ax.text(5.5, 1, "Oligomycin", ha="center", va="center", rotation=-90, size=16, bbox=Dict("boxstyle" => "rarrow", "fc" => "w", "ec" => "k", "lw" => 2, "alpha" => 0.5))
            elseif row == 2
                ax.text(5.5, 1, "Rotenone", ha="center", va="center", rotation=-90, size=16, bbox=Dict("boxstyle" => "rarrow", "fc" => "w", "ec" => "k", "lw" => 2, "alpha" => 0.5))
            elseif row == 3
                ax.text(5.5, 2.5, "FCCP", ha="center", va="center", rotation=90, size=16, bbox=Dict("boxstyle" => "rarrow", "fc" => "w", "ec" => "k", "lw" => 2, "alpha" => 0.5))
            end
            cbar = fig.colorbar(mesh, ax=ax)
            cbar.ax.set_title(cbl)
        end
    end

    plt.tight_layout()
    return fig
end

In [ ]:
fig3 = plot_fig3()

In [ ]:
# Uncomment to generate the pdf file 
fig3.savefig("Fig3.tif", dpi=300, format="tiff", pil_kwargs=Dict("compression" => "tiff_lzw"))

## Free fatty acid and galactose treatment

- Free fatty acid: adding more citric acid cycle (CAC) flux
- Galactose: Tweak the stoichiometry of glucokinase (GK) so that glycolysis yields no ATP.

In [ ]:
using MitochondrialDynamics: GlcConst, G3P, Pyr, NADH_c, NADH_m, ATP_c, ADP_c, AMP_c, Ca_m, Ca_c, x, ΔΨm, degavg, J_CAC

In [ ]:
prob = SteadyStateProblem(sys, []) # Use default u0
sol = solve(prob)

glc = range(3.0mM, 30.0mM, length=101)  # Range of glucose

function remake_glc(prob, g)
    p = copy(prob.p)
    p[idxGlc] = g
    remake(prob; p=p)
end

sols = map(glc) do g
    solve(remake_glc(prob, g), DynamicSS(Rodas5()))
end

# Additional CAC flux: 3.4 μM/s
@named sysffa = make_model(j_ffa=sol[J_CAC] * 0.5)
probffa = SteadyStateProblem(sysffa, [])
solsffa = map(glc) do g
    solve(remake_glc(probffa, g), DynamicSS(Rodas5()))
end

@named sys_gal = make_model(gk_atp_stoich=4)
prob_gal = SteadyStateProblem(sys_gal, [])
sols_gal = map(glc) do g
    solve(remake_glc(prob_gal, g), DynamicSS(Rodas5()))
end;

In [ ]:
function plot_ffa(
    sols, solsffa, glc;
    figsize=(12, 12),
    tight=true,
    labels=["Default", "FFA"]
)
    glc5 = glc ./ 5
    fig, ax = plt.subplots(3, 3; figsize)

    ax[1, 1].plot(glc5, extract(sols, G3P) .* 1000, label=labels[1])
    ax[1, 1].plot(glc5, extract(solsffa, G3P) .* 1000, label=labels[2])
    ax[1, 1].set(title="(A) G3P (μM)", ylim=(0, 10))

    ax[1, 2].plot(glc5, extract(sols, Pyr) .* 1000, label=labels[1])
    ax[1, 2].plot(glc5, extract(solsffa, Pyr) .* 1000, label=labels[2])
    ax[1, 2].set(title="(B) Pyruvate (μM)", ylim=(0, 160))

    ax[1, 3].plot(glc5, extract(sols, Ca_c) .* 1000, label=labels[1])
    ax[1, 3].plot(glc5, extract(solsffa, Ca_c) .* 1000, label=labels[2])
    ax[1, 3].set(title="(C) Cytosolic Calcium (μM)", ylim=(0.0, 0.4))

    ax[2, 1].plot(glc5, extract(sols, Ca_m) .* 1000, label=labels[1])
    ax[2, 1].plot(glc5, extract(solsffa, Ca_m) .* 1000, label=labels[2])
    ax[2, 1].set(title="(D) Mitochondrial Calcium (μM)", ylim=(0.0, 1.5))

    ax[2, 2].plot(glc5, extract(sols, NADH_c) .* 1000, label=labels[1])
    ax[2, 2].plot(glc5, extract(solsffa, NADH_c) .* 1000, label=labels[2])
    ax[2, 2].set(title="(E) Cytosolic NADH (μM)")

    ax[2, 3].plot(glc5, extract(sols, NADH_m) .* 1000, label=labels[1])
    ax[2, 3].plot(glc5, extract(solsffa, NADH_m) .* 1000, label=labels[2])
    ax[2, 3].set(title="(F) Mitochondrial NADH (μM)")

    ax[3, 1].plot(glc5, extract(sols, ATP_c) ./ extract(sols, ADP_c), label=labels[1])
    ax[3, 1].plot(glc5, extract(solsffa, ATP_c) ./ extract(solsffa, ADP_c), label=labels[2])
    ax[3, 1].set(title="(G) ATP:ADP")

    ax[3, 2].plot(glc5, extract(sols, ΔΨm) .* 1000, label=labels[1])
    ax[3, 2].plot(glc5, extract(solsffa, ΔΨm) .* 1000, label=labels[2])
    ax[3, 2].set(title="(H) ΔΨ (mV)")

    ax[3, 3].plot(glc5, extract(sols, degavg), label=labels[1])
    ax[3, 3].plot(glc5, extract(solsffa, degavg), label=labels[2])
    ax[3, 3].set(title="(I) Average Degree")

    for a in ax
        a.set_xticks(1:6)
        a.grid()
        a.legend()
    end

    fig.set_tight_layout(tight)
    return fig
end

In [ ]:
fig_ffa = plot_ffa(sols, solsffa, glc; labels=["Default", "FFA"])

In [ ]:
# Uncomment if pdf file is required
fig_ffa.savefig("Fig_FFA.tif", dpi=300, format="tiff", pil_kwargs=Dict("compression" => "tiff_lzw"))

In [ ]:
fig_gal = plot_ffa(sols, sols_gal, glc; labels=["Default", "Galactose"])

In [ ]:
# Uncomment if pdf file is required
fig_gal.savefig("Fig_GAL.tif", dpi=300, format="tiff", pil_kwargs=Dict("compression" => "tiff_lzw"))